<a href="https://colab.research.google.com/github/SathishMalaVish/BookScape-Explorer/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Set your ngrok auth token
from pyngrok import ngrok
ngrok.set_auth_token("2qZXTI70wbnYK9o4Ldi7bDiVMwX_6N2MUv8REzFnRsePg6zhh")


# Start ngrok tunnel
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

# Run the Streamlit app
!streamlit run book.py &>/dev/null&

Streamlit app is live at: NgrokTunnel: "https://4e16-34-90-193-245.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pip install streamlit
!pip install mysql-connector-python
!pip install pyngrok
!npm install localtunnel
!pip install ngrok
!pip install mysql
!pip install requests pandas pymysql streamlit
!pip install streamlit-lottie